# Chapter 9: Investment Projections
In this chapter you will learn how to calculate and visualize projections for investments using our Model Portfolios. We will show possible outcomes given certain investment time horizons and amounts.

In [ ]:
import yfinance as yf

In [ ]:
#HEADING 2: Calculating investment projections
#SKILL 2: Learn how to calculate investment projections

# Future Value (Compound)
#expectedReturn = myPortfolio.expectedReturn
expectedReturn = 0.08
initialInvestment = 5000
years = 1

valuePrincipal = initialInvestment * (1 + expectedReturn)
print(valuePrincipal)

valuePrincipal = initialInvestment * pow(1 + expectedReturn/12, (years*12))
print(valuePrincipal)

In [ ]:
# Future Value Annuity (Compound)
monthlyInvestment = 100
valueMonthly = monthlyInvestment * (pow(1 + expectedReturn/12, (years*12))-1)/(expectedReturn/12)
print(valueMonthly)

In [ ]:
# Combined future value
totalValue = valuePrincipal+valueMonthly
print(totalValue)

In [ ]:
def returnProjection(expectedReturn, initialInvestment, monthlyInvestment, years):
  valuePrincipal = initialInvestment * pow(1 + expectedReturn/12, (years*12))
  valueMonthly = monthlyInvestment * (pow(1 + expectedReturn/12, (years*12))-1)/(expectedReturn/12)
  return valuePrincipal+valueMonthly

In [ ]:
expectedRisk = 0.10
expectedReturnMin = expectedReturn - expectedRisk
expectedReturnMax = expectedReturn + expectedRisk
print(expectedReturnMin)
print(returnProjection(expectedReturnMin, initialInvestment, monthlyInvestment, years))
print(expectedReturnMax)
print(returnProjection(expectedReturnMax, initialInvestment, monthlyInvestment, years))

In [ ]:
#HEADING 3: Visualizing investment projections
#SKILL 3: Learn how to visualize investment projections

In [ ]:
def returnProjectionByYear(expectedReturn, expectedRisk, initialInvestment, monthlyInvestment, years):
  from datetime import date
  import pandas as pd
  df = pd.DataFrame({'date': [],
                   'lowValue': [],
                   'value': [],
                   'highValue': []})
  df.set_index('date')

  for year in range(years+1):
    newValue = returnProjection(expectedReturn, initialInvestment, monthlyInvestment, year)
    newValueLower = returnProjection(expectedReturn-expectedRisk, initialInvestment, monthlyInvestment, year)
    newValueUpper = returnProjection(expectedReturn+expectedRisk, initialInvestment, monthlyInvestment, year)
    newDate = date.today()
    newDate = newDate.replace(year=newDate.year + year)
    df = df.append(pd.Series({'date': newDate, 'lowValue': newValueLower, 'value': newValue, 'highValue': newValueUpper},name=''))
  
  df = df.set_index(pd.DatetimeIndex(df['date']))
  df = df.drop(columns="date")
  return df

In [ ]:
data = returnProjectionByYear(expectedReturn, expectedRisk, initialInvestment, monthlyInvestment, years)
data


In [ ]:
years = 10
data = returnProjectionByYear(expectedReturn, expectedRisk, initialInvestment, monthlyInvestment, years)
data

In [ ]:
import matplotlib.pyplot as plt
plt.plot(data.index, data['highValue'], label="High")
plt.plot(data.index, data['value'], label="Expected")
plt.plot(data.index, data['lowValue'], label="Low")
plt.legend(loc="upper left")
plt.show()

In [ ]:
years = 30
data = returnProjectionByYear(expectedReturn, expectedRisk, initialInvestment, monthlyInvestment, years)

In [ ]:
plt.plot(data.index, data['highValue'], label="High")
plt.plot(data.index, data['value'], label="Expected")
plt.plot(data.index, data['lowValue'], label="Low")
plt.legend(loc="upper left")
plt.show()

In [ ]:
class Projection:
  def __init__(self, expectedReturn: float, expectedRisk: float, initialInvestment: float, monthlyInvestment: float, years: int):
    from datetime import date
    import pandas as pd
    df = pd.DataFrame({'date': [],
                    'lowValue': [],
                    'value': [],
                    'highValue': []})
    df.set_index('date')

    for year in range(years+1):
      newValue = self.returnProjection(expectedReturn, initialInvestment, monthlyInvestment, year)
      newValueLower = self.returnProjection(expectedReturn-expectedRisk, initialInvestment, monthlyInvestment, year)
      newValueUpper = self.returnProjection(expectedReturn+expectedRisk, initialInvestment, monthlyInvestment, year)
      newDate = date.today()
      newDate = newDate.replace(year=newDate.year + year)
      df = df.append(pd.Series({'date': newDate, 'lowValue': newValueLower, 'value': newValue, 'highValue': newValueUpper},name=''))
    
    df = df.set_index(pd.DatetimeIndex(df['date']))
    df = df.drop(columns="date")
    self.data = df

  @staticmethod
  def returnProjection(expectedReturn, initialInvestment, monthlyInvestment, years):
    valuePrincipal = initialInvestment * pow(1 + expectedReturn/12, (years*12))
    valueMonthly = monthlyInvestment * (pow(1 + expectedReturn/12, (years*12))-1)/(expectedReturn/12)
    return valuePrincipal+valueMonthly

  def visualize(self, targetAmount: float = 0.0):
    import matplotlib.pyplot as plt
    import matplotlib.ticker as ticker
    scale_y = 1e6
    ticks_y = ticker.FuncFormatter(lambda x, pos: '{0:g}'.format(x/scale_y))
    fig, ax=plt.subplots()
    ax.yaxis.set_major_formatter(ticks_y)
    ax.set_ylabel('Millions (USD)')
    ax.plot(self.data.index, self.data['highValue'], label="High")
    ax.plot(self.data.index, self.data['value'], label="Expected")
    ax.plot(self.data.index, self.data['lowValue'], label="Low")
    plt.legend(loc="upper left")
    if (targetAmount > 0):
      plt.axhline(y=targetAmount)
    plt.show()

In [ ]:
proj = Projection(expectedReturn, expectedRisk, initialInvestment, monthlyInvestment, years)
proj.visualize()

In [ ]:
class Goal:
  def __init__(self, name, targetYear, targetValue, initialContribution=0, monthlyContribution=0, priority=""):
    self.name = name
    self.targetYear = targetYear
    self.targetValue = targetValue
    self.initialContribution = initialContribution
    self.monthlyContribution = monthlyContribution
    if not (priority == "") and not (priority in ["Dreams", "Wishes", "Wants", "Needs"]):
            raise ValueError('Wrong value set for Priority.')
    self.priority = priority

  def getGoalProbabilities(self):
    if (self.priority == ""):
            raise ValueError('No value set for Priority.')
    import pandas as pd
    lookupTable=pd.read_csv('./Data/Goal Probability Table.csv')
    match = (lookupTable['Realize'] == self.priority)
    minProb = lookupTable['MinP'][(match)]
    maxProb = lookupTable['MaxP'][(match)]
    return minProb.values[0], maxProb.values[0]

In [ ]:
myGoal = Goal("Retirement", 
              targetYear=2041, 
              targetValue=3000000, 
              initialContribution=50000, 
              monthlyContribution=500, 
              priority="Wishes")

In [ ]:
#HEADING 3: Calculating a Risk Score

# Interactive questionnaire

class RiskQuestion:
  def __init__(self, questionText, weight=1):
    self.questionText = questionText
    self.weight = weight
    self.answers = []

class RiskQuestionAnswer:
  def __init__(self, answerText, score, selected=False):
    self.answerText = answerText
    self.score = score
    self.selected = selected

class RiskQuestionnaire:
  def __init__(self):
    self.questions = []
    self.score = 0

  def loadQuestionnaire(self, riskQuestionsFileName, riskAnswersFileName, type):

    if not (type in ["Tolerance", "Capacity"]):
            raise ValueError('Type must be Tolerance or Capacity.')

    import pandas as pd
    riskQuestions = pd.read_csv(riskQuestionsFileName).reset_index()
    riskAnswers = pd.read_csv(riskAnswersFileName).reset_index()

    if (type == "Tolerance"):
      toleranceQuestions = riskQuestions[(riskQuestions['QuestionType'] == 'Tolerance')].reset_index()
      for index, row in toleranceQuestions.iterrows():
          self.questions.append(RiskQuestion(row['QuestionText'], row['QuestionWeight']))
          answers = riskAnswers[(riskAnswers['QuestionID'] == row['QuestionID'])]
          for indexA, rowA in answers.iterrows():
                self.questions[index].answers.append(RiskQuestionAnswer(rowA['AnswerText'],rowA['AnswerValue']))
    else:
      capacityQuestions = riskQuestions[(riskQuestions['QuestionType'] == 'Capacity')].reset_index()
      for index, row in capacityQuestions.iterrows():
          self.questions.append(RiskQuestion(row['QuestionText'], row['QuestionWeight']))
          answers = riskAnswers[(riskAnswers['QuestionID'] == row['QuestionID'])]
          for indexA, rowA in answers.iterrows():
                self.questions[index].answers.append(RiskQuestionAnswer(rowA['AnswerText'],rowA['AnswerValue']))
    

  def answerQuestionnaire(self):
    for i in range(len(self.questions)):
      question = self.questions[i]
      print(question.questionText)
      for n in range(len(question.answers)):
        answer = question.answers[n]
        print(str(n) + ": " + answer.answerText)
      nChosen = int(input("Choose your answer between 0 and " + str(len(question.answers)-1) + ": "))
      self.questions[i].answers[nChosen].selected = True
      print("\n")

  def calculateScore(self):
    print("Risk Score:")
    myTotalScore = 0
    for question in self.questions:
      for answer in question.answers:
        if (answer.selected == True):
          myTotalScore = myTotalScore + (answer.score * question.weight)
          print(answer.answerText + ": " + str(answer.score * question.weight))
    print("Total Risk Score: " + str(myTotalScore) + "\n")
    self.score = myTotalScore

In [ ]:
questionsFileName = './Data/Risk Questions.csv'
answersFileName = './Data/Risk Answers.csv'

toleranceQuestionnaire = RiskQuestionnaire()
toleranceQuestionnaire.loadQuestionnaire(questionsFileName, answersFileName, "Tolerance")

capacityQuestionnaire = RiskQuestionnaire()
capacityQuestionnaire.loadQuestionnaire(questionsFileName, answersFileName, "Capacity")

toleranceQuestionnaire.answerQuestionnaire()
capacityQuestionnaire.answerQuestionnaire()

In [ ]:
toleranceQuestionnaire.calculateScore()
capacityQuestionnaire.calculateScore()
riskTolScore = toleranceQuestionnaire.score
riskCapScore = capacityQuestionnaire.score

In [ ]:
class Portfolio:

  def __init__(self, tickerString: str, expectedReturn: float, portfolioName: str, riskBucket: int):

    self.name = portfolioName
    self.riskBucket = riskBucket
    self.expectedReturn = expectedReturn
    self.allocations = []

    from pypfopt.efficient_frontier import EfficientFrontier
    from pypfopt import risk_models
    from pypfopt import expected_returns

    df = self.__getDailyPrices(tickerString, "20y")

    mu = expected_returns.mean_historical_return(df)
    S = risk_models.sample_cov(df)

    ef = EfficientFrontier(mu, S)

    ef.efficient_return(expectedReturn)
    self.expectedRisk = ef.portfolio_performance()[1]
    portfolioWeights = ef.clean_weights()

    for key, value in portfolioWeights.items():
      newAllocation = Allocation(key, value)
      self.allocations.append(newAllocation)

  def __getDailyPrices(self, tickerStringList, period):
    data = yf.download(tickerStringList, group_by="Ticker", period=period)
    data = data.iloc[:, data.columns.get_level_values(1)=="Close"]
    data = data.dropna()
    data.columns = data.columns.droplevel(1)
    return data

  def printPortfolio(self):
    print("Portfolio Name: " + self.name)
    print("Risk Bucket: " + str(self.riskBucket))
    print("Expected Return: " + str(self.expectedReturn))
    print("Expected Risk: " + str(self.expectedRisk))
    print("Allocations: ")
    for allocation in self.allocations:
      print("Ticker: " + allocation.ticker + ", Percentage: " + str(allocation.percentage))

  @staticmethod
  def getPortfolioMapping(riskToleranceScore, riskCapacityScore):
    import pandas as pd
    allocationLookupTable=pd.read_csv('./Data/Risk Mapping Lookup.csv')
    matchTol = (allocationLookupTable['Tolerance_min'] <= riskTolScore) & (allocationLookupTable['Tolerance_max'] >= riskTolScore)
    matchCap = (allocationLookupTable['Capacity_min'] <= riskCapScore) & (allocationLookupTable['Capacity_max'] >= riskCapScore)
    portfolioID = allocationLookupTable['Portfolio'][(matchTol & matchCap)]
    return portfolioID.values[0]

In [ ]:
myPortfolioID = Portfolio.getPortfolioMapping(riskTolScore, riskCapScore)

myPortfolio = Portfolio("VTI TLT IEI GLD DBC", expectedReturn = 0.06, portfolioName = "Moderate Growth", riskBucket = myPortfolioID)
myPortfolio.printPortfolio()

In [ ]:
from datetime import date
yearsToGoal = myGoal.targetYear - date.today().year
myProjection = Projection(myPortfolio.expectedReturn, 
                          expectedRisk=myPortfolio.expectedRisk, 
                          initialInvestment=myGoal.initialContribution, 
                          monthlyInvestment=myGoal.monthlyContribution, 
                          years=yearsToGoal)
myProjection.visualize(myGoal.targetValue)

In [ ]:
# Check if goal amount is achieved with expectedReturns by goalTimeline
import pandas as pd
def checkGoalPlausible(df: pd.DataFrame, goalValue) -> bool:
  maxValue = df['value'].max()
  if maxValue >= goalValue:
    return True
  else:
    return False

In [ ]:
checkGoalPlausible(myProjection.data, myGoal.targetValue)

In [ ]:
import scipy.stats as st
import math as math

minReturn = 0.022
avgReturn = 0.05
avgRisk = 0.07
timeHorizon = 10
std = avgRisk/math.sqrt(timeHorizon)
print(std)

z_score = (minReturn-avgReturn)/std
print(z_score)
print(1-st.norm.cdf(z_score))

# Use this it works! Need to add goal priority and use probs to determine if feasible. Can add risk or cash?

In [ ]:
# TODO: Use goal.priority, plus method to check if probability is in right range? If too high, take more risk? If too low, save more or extend timeline?
myGoal.getGoalProbabilities()

In [ ]:
# Many ways to calculate, this shows probability goal target amount will be exceeded over goal timeline, ideally above 50%
def goalProbability(minReturn, avgReturn, avgRisk, timeHorizon) -> float:
  import scipy.stats as st
  std = avgRisk/math.sqrt(timeHorizon)
  z_score = (minReturn-avgReturn)/std
  return 1-st.norm.cdf(z_score)

In [ ]:
goalProbability(minReturn, avgReturn, avgRisk, timeHorizon)

In [ ]:
# Many ways to calculate, this shows probability goal target amount will be exceeded over goal timeline, ideally above 50%
import pandas as pd
def goalProbabilityForAmount(goalAmount, expectedReturn, portfolioRisk, years, initialInvestment, monthlyInvestment) -> float:
  import scipy.stats as st
  import math as math
  std = portfolioRisk/math.sqrt(years)
  
  amount = 0
  minReturn = 0.00
  while (amount < goalAmount):
    minReturn = minReturn + 0.0000001
    amount = Projection.returnProjection(minReturn, initialInvestment, monthlyInvestment, years)
  
  z_score = (minReturn-expectedReturn)/std
  return 1-st.norm.cdf(z_score)

In [ ]:
goalProbabilityForAmount(myGoal.targetValue, 
                         myPortfolio.expectedReturn, 
                         myPortfolio.expectedRisk, 
                         yearsToGoal, 
                         myGoal.initialContribution, 
                         myGoal.monthlyContribution)

In [ ]:
# Reverse calculate from FV formula required monthly investment to hit goal amount by goalTimeline?
def calculateMonthlyMinimum(expectedReturn, initialInvestment, years, goalAmount) -> float:
  monthlyInvestment = (goalAmount - (initialInvestment * pow(1 + expectedReturn/12, (years*12))))/((pow(1 + expectedReturn/12, (years*12))-1)/(expectedReturn/12)) 
  return monthlyInvestment

In [ ]:
calculateMonthlyMinimum(myPortfolio.expectedReturn, 
                        myGoal.initialContribution, 
                        yearsToGoal, 
                        myGoal.targetValue)

In [ ]:
myGoal.monthlyContribution = calculateMonthlyMinimum(myPortfolio.expectedReturn, 
                                                     myGoal.initialContribution, 
                                                     yearsToGoal, 
                                                     myGoal.targetValue)

goalProbabilityForAmount(myGoal.targetValue, 
                         myPortfolio.expectedReturn, 
                         myPortfolio.expectedRisk, 
                         yearsToGoal, 
                         myGoal.initialContribution, 
                         myGoal.monthlyContribution)

In [ ]:
myProjection = Projection(myPortfolio.expectedReturn, 
                          myPortfolio.expectedRisk, 
                          myGoal.initialContribution, 
                          myGoal.monthlyContribution, 
                          yearsToGoal)
myProjection.visualize(myGoal.targetValue)

In [ ]:
goalProbabilityForAmount(myGoal.targetValue, 
                         myPortfolio.expectedReturn, 
                         myPortfolio.expectedRisk, 
                         yearsToGoal+2, 
                         myGoal.initialContribution, 
                         myGoal.monthlyContribution)

In [ ]:
myProjection = Projection(myPortfolio.expectedReturn, 
                          myPortfolio.expectedRisk, 
                          myGoal.initialContribution, 
                          myGoal.monthlyContribution, 
                          yearsToGoal+2)
myProjection.visualize(myGoal.targetValue)

In [ ]:
goalProbabilityForAmount(myGoal.targetValue, 
                         myPortfolio.expectedReturn, 
                         myPortfolio.expectedRisk, 
                         30, 
                         500000, 
                         1500)

In [ ]:
pip install pandas_montecarlo --upgrade --no-cache-dir

In [ ]:
import pandas_montecarlo
data = yf.download("VTI TLT IEI GLD DBC", group_by="Ticker", period="20y")
data = data.iloc[:, data.columns.get_level_values(1)=="Close"]
data = data.dropna()
data.columns = data.columns.droplevel(1)
data['Total'] = data.sum(axis=1)
data['Return'] = data['Total'].pct_change().fillna(0)
data

In [ ]:
mc = data['Return'].montecarlo(sims=100)
mc.plot(title="Portfolio Returns Monte Carlo Simulations", figsize=(12,7))